In [1]:
import pandas as pd
from metasynth import MetaDataset
import wget
from pathlib import Path
import json

In [2]:
dtypes = {
    "Survived": "category",
    "Pclass": "category",
    "Name": "string",
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category"
}

In [3]:
titanic_fp = Path("titanic.csv")
if not titanic_fp.is_file():
    wget.download("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")

In [4]:
df = pd.read_csv(titanic_fp, dtype=dtypes)

In [5]:
dataset = MetaDataset.from_dataframe(df)

In [6]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,<NA>,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,<NA>,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,<NA>,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,<NA>,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,<NA>,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [13]:
dataset.synthesize(1000)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,1,3,">CMdf_= h]tv tH+ZThbmP( Ff3 +@G`-iR2yl,}66$X4d...",male,23.854665,0,2,Ry|3,21.296547,<NA>,S
1,2,1,2,J8r}X#y#d66?[BKWTXu/QS%_iU (y7k8,male,47.219963,0,0,789,1.755228,<NA>,S
2,3,0,3,"H%cM0}KDou~w:[TJ!G;>ez+ =,Qrcd fwR8sa[",male,18.697817,0,2,SQ[$hq49,39.008204,<NA>,S
3,4,1,3,";[m`rCk`R]Ddfh}]p:bjg>Y+""p=dR-9\B%uo97y%=}N> b...",male,24.181034,1,0,11580,11.780319,<NA>,S
4,5,0,3,"r>G`<vl""B`3.3 2^Yflf c n;5fpzUl7F--o^|.1[}%B^n...",male,33.767196,0,0,53470,66.502671,<NA>,C
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,0,2,0'Y @Xtb WZzcWb7 i,male,54.833319,1,0,773921,0.571822,<NA>,S
996,997,1,3,"j\jdm[""#P3=n5q",female,35.188650,0,1,79738,0.890048,<NA>,S
997,998,0,1,@T&-=O1Da%fI_g1NBGzc)b* Qm 4G7GLg(K=H<a''Bnbpp...,female,23.566370,0,0,2621,11.878882,<NA>,S
998,999,1,3,"h""M$""k""FuE 81",male,39.806907,0,0,411270,93.644010,<NA>,S


In [8]:
dataset.to_json("test.json")

In [9]:
print(MetaDataset.from_json("test.json"))

# Rows: 891
# Columns: 12

{'name': 'PassengerId', 'type': 'IntVar', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'UniqueKeyDistribution', 'parameters': {'low': 1, 'consecutive': 1}}"}

{'name': 'Survived', 'type': 'CategoricalVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'CatFreqDistribution', 'parameters': {'categories': ['0', '1'], 'counts': [549, 342]}}"}

{'name': 'Pclass', 'type': 'CategoricalVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'CatFreqDistribution', 'parameters': {'categories': ['1', '2', '3'], 'counts': [216, 184, 491]}}"}

{'name': 'Name', 'type': 'StringVar', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,81}[)][ ]'}

{'name': 'Sex', 'type': 'CategoricalVar', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'CatFreqDistribution', 'parameters': {'categories': ['female', 'male'], 'counts': [314, 577]}}"}

{'name': 'Age', 'type': 'FloatVar', 'dtype': 'float64

In [11]:
dataset.to_dict()

{'n_rows': 891,
 'n_columns': 12,
 'vars': [{'name': 'PassengerId',
   'type': 'IntVar',
   'dtype': 'int64',
   'prop_missing': 0.0,
   'distribution': {'name': 'UniqueKeyDistribution',
    'parameters': {'low': 1, 'consecutive': 1}}},
  {'name': 'Survived',
   'type': 'CategoricalVar',
   'dtype': 'category',
   'prop_missing': 0.0,
   'distribution': {'name': 'CatFreqDistribution',
    'parameters': {'categories': array(['0', '1'], dtype='<U1'),
     'counts': array([549, 342])}}},
  {'name': 'Pclass',
   'type': 'CategoricalVar',
   'dtype': 'category',
   'prop_missing': 0.0,
   'distribution': {'name': 'CatFreqDistribution',
    'parameters': {'categories': array(['1', '2', '3'], dtype='<U1'),
     'counts': array([216, 184, 491])}}},
  {'name': 'Name',
   'type': 'StringVar',
   'dtype': 'string',
   'prop_missing': 0.0,
   'distribution': {'name': 'RegexDistribution',
    'parameters': {'re_list': [('.[]{12,81}', 1.0),
      ('[)]', 0.16049382716049382),
      ('[ ]', 0.0011223

In [12]:
print(MetaDataset.from_xml("test.xml"))

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [ ]:
dataset.to_dict()